In [136]:
!pip install xlsxwriter -q

In [137]:
%load_ext autoreload
%autoreload 2
    
%matplotlib inline

#basic data analysis libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pylab as plt
import seaborn as sns

import os, glob
import re
from tqdm import tqdm

from IPython.display  import HTML

%config InlineBackend.figure_format='retina'
np.set_printoptions(precision=4)
pd.options.display.max_rows = 999
pd.options.display.float_format = '{:,.3f}'.format
sns.set_style("whitegrid")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
def drop_cols(df, cols_to_drop):
    cols_to_drop = [c for c in cols_to_drop if c in df.columns];
    if len(cols_to_drop)>0:
        df.drop(columns=cols_to_drop, inplace=True);
    print(f'Colunas atuais {len(df.columns):<4} - Colunas Removidas {len(cols_to_drop):<4}')
    return df;

In [139]:
def filterCols(df, subs):
    cols = [c for c in df.columns if subs in c];
    return df[cols]

In [140]:
df_pesquisa = (
    pd.read_excel('../data/k24069946_240627_102351_-03.xlsx')
)

## Removendo colunas não uteis ou com preenchimento constante ou que sejam de controle

In [141]:
colunas_removidas = []

In [142]:
# removendo colunas 100% em branco
miss100 = df_pesquisa.isna().mean().to_frame('pmiss').query('pmiss ==1').index.tolist()
df_pesquisa.pipe(drop_cols,miss100)

colunas_removidas += [ (c,'100% vazia') for c in miss100]

Colunas atuais 1744 - Colunas Removidas 741 


In [143]:
# Removendo colunas com valores contantes
valores_unicos_1 = df_pesquisa.nunique().to_frame('valores_unicos').query('valores_unicos ==1').index.tolist()
df_pesquisa.pipe(drop_cols,valores_unicos_1)
colunas_removidas += [ (c,'Valores constantes') for c in valores_unicos_1]

Colunas atuais 986  - Colunas Removidas 758 


In [144]:
# removendo colunas de controle para preenchimento de cotas da pesquisa
cota_cols  = [c for c in df_pesquisa.columns.tolist() if 'QGEN_AGE_QTA' in c or 'QCURRENT_QTA' in c or 'QSPRINCIPAL_QTA' in c] 
cota_cols += ['QREGION_QTA','QSINC_QTA']
df_pesquisa.pipe(drop_cols,cota_cols)
colunas_removidas += [ (c,'flags cotas') for c in cota_cols]

Colunas atuais 909  - Colunas Removidas 77  


In [145]:
cota_cols = [c for c in df_pesquisa.columns.tolist() if 'QCOUNTRY_GENAGE' in c or 'QBANKED_DEFAULTER' in c or 'QCOUNTRY_X_DEFAULTER' in c]
df_pesquisa.pipe(drop_cols,cota_cols)
colunas_removidas += [ (c,'flags cotas') for c in cota_cols]

Colunas atuais 880  - Colunas Removidas 29  


In [146]:
# removendo colunas "outros" com baixo preenchimento
col_others = [c for c in df_pesquisa.columns.tolist() if ('QPRODUCT_' in c and 'OTH' in c) or 'QATTCHECKS' in c]
col_others += [
    'QDWORK_97_OTH','QSESCOL_97_OTH','Q30R97_OTH','Q34R97_OTH','Q25R97_OTH','Q44R97_OTH','Q45R97_OTH',
    'Q13.9','Q38.97','Q23_97_OTH','Q38_97_OTH','Q39.97','Q39_97_OTH','Q40.97','Q40_97_OTH',]
df_pesquisa.pipe(drop_cols,col_others)
colunas_removidas += [ (c,'Outros') for c in col_others]

Colunas atuais 844  - Colunas Removidas 36  


In [147]:
colunas_extras  = ['resp_last_ts','QREV','QSAMP','QGENDER','QBANK_SAMP','QDEFAULTER_SAMP']
colunas_extras += ['tz_name', 'win_version','failed_attempts','QSURTM', 'QFEEDBACK_english_us','QAGEDIFF',
     'timezone','browser_version','screensize','device_type',]

df_pesquisa.pipe(drop_cols,colunas_extras)
colunas_removidas += [ (c,'Irrelevantes') for c in colunas_extras]

Colunas atuais 828  - Colunas Removidas 16  


In [148]:
(
    pd.DataFrame(colunas_removidas, columns=['col','reason2remove'])
    .groupby('reason2remove')
    .count()
)

,col
reason2remove,
100% vazia,741
Irrelevantes,16
Outros,37
Valores constantes,758
flags cotas,106


In [149]:
def moveColuns(df, cols_first=[], cols_last=[]):
    """"
    Reorder the coluns moving to the first position, some to the last and the rest in the middle
    """
    columns = df.columns;
    if len(cols_first)>1:
        cols_first = [c for c in columns if c in cols_first]

    if len(cols_last) > 1:
        cols_last = [c for c in columns if c in cols_last]

    others = [c for c in columns if c not in cols_first and c not in cols_last]

    return df[cols_first+others+cols_last];

In [150]:
def cleanBanc(row, others):
    """
        Alguns clientes preencheram que tem outros bancos porém na descrição eles listam bancos que deveria estar na lista.
        Essa funcao visa reduzir o número de outros
    """
    if row['QSPRINCIPAL'] < 500:
        return row['QSPRINCIPAL'];
    else:
        v = row[others]
        if type(v) != float:
            v = v.upper().strip()
            if v == 'NUBANK':return 222;
            elif 'SANTANDER' in v: return 226;
            elif 'ITA' in v: return 217;
            elif 'CAIXA' in v: return 211;
            elif 'BRADESCO' in v: return 207;
            elif 'ORIGINAL' in v: return 204;
            elif 'INTER' in v: return 216;
            elif 'BRASIL' in v: return 203;
            elif 'MERCADO' in v: return 219;
            elif 'PAG' in v and 'BANK' in v: return 223;
            elif 'NEON' in v: return 220;
            elif 'SICOOB' in v:return 227;
            elif 'SICRED' in v:return 228;
            elif 'PICPAY' in v:return 224;
    

    return 288;

In [151]:
def cleanFeedBack(df):
    """
        clean if the feedback has few words
    """
    df = df.copy()
    df['QFEEDBACK'] = df['QFEEDBACK'].apply(lambda x: x if type(x) == str and len(x.split())>5 else '' )
    return df;

# main loop

In [152]:
from mappings2 import *

depara_colunas = []
drop_after_processing = ['QSPRINCIPAL_971_OTH','QSPRINCIPAL_972_OTH','QSPRINCIPAL_973_OTH',]

renames_basicos = {
    'resp_start_ts':'data_resposta',
    'QSAGE':'idade',
    'QAGEMATH':'idade_calculada',
    'QAGE':'faixa_idade',
    'QDYEARBORN':'ano_nascimento',
    'Q13_97_OTH':'origem_dinheiro_pagamento_contas__outros',

}

for k,v in renames_basicos.items():
    depara_colunas.append([k,v])
    print(f"{k:<20} ==> {v}")

df_transf = (
    df_pesquisa
    .copy()
    .assign(resp_start_ts=lambda x:x['resp_start_ts'].dt.date)
    .assign(QSPRINCIPAL=lambda x:x[['QSPRINCIPAL','QSPRINCIPAL_971_OTH']].apply(cleanBanc,others='QSPRINCIPAL_971_OTH', axis=1))
    .assign(QSPRINCIPAL=lambda x:x[['QSPRINCIPAL','QSPRINCIPAL_972_OTH']].apply(cleanBanc,others='QSPRINCIPAL_972_OTH', axis=1))
    .assign(QSPRINCIPAL=lambda x:x[['QSPRINCIPAL','QSPRINCIPAL_973_OTH']].apply(cleanBanc,others='QSPRINCIPAL_973_OTH', axis=1))

    .pipe(cleanFeedBack)
    .rename(columns=renames_basicos)
    .drop(columns=drop_after_processing)
)

for map_ in mapeamento:
    col = map_['col']
    rename= map_['rename']
    map = map_['map']
    depara_colunas.append([col, rename])
    print(f"{col:<20} ==> {rename}")

    if col not in df_transf.columns:
        print(f"+{col} não encontrada.")
        #print(map_)
        continue;
        

    df_transf = (
        df_transf
        .rename(columns={col:rename})
        .assign(**{
            rename:lambda df: df[rename].map(map)
        })
    )


df_transf=df_transf.pipe(moveColuns,cols_first=[
    'id','data_resposta','elapsed','genero','idade','idade_calculada','faixa_idade','ano_nascimento','escolaridade','estado_civil',
    'estado','regiao','capital','tipo_emprego',
    'banco_principal','default','default_detalhe',
    'cartao_loja','consistencia_pagamento_cartao','comprometimento_renda_com_parcelas',
    'perspectiva_futuro','nivel_suporte_contratacao', 'transparencia_comunicacao',
], cols_last=['QFEEDBACK','QATT1R88','Q13.97','Q13_97_OTH','Q44R97_OTH','Q45R97_OTH','Q26R88'])

resp_start_ts        ==> data_resposta
QSAGE                ==> idade
QAGEMATH             ==> idade_calculada
QAGE                 ==> faixa_idade
QDYEARBORN           ==> ano_nascimento
Q13_97_OTH           ==> origem_dinheiro_pagamento_contas__outros
QSGEND               ==> genero
QSSTATE              ==> estado
QREGION              ==> regiao
QSCAPITAL            ==> capital
QDMARITAL            ==> estado_civil
QSINC                ==> faixa_renda
QSESCOL              ==> escolaridade
QDWORK               ==> tipo_emprego
QNB0                 ==> bancarizado_uso_conta
QBANKED              ==> ind_bancarizado
QNB1                 ==> cartao_loja
QSPRINCIPAL          ==> banco_principal
QDEFAULTER           ==> default
QINA                 ==> default_detalhe
QFUT                 ==> perspectiva_futuro
Q19                  ==> nivel_suporte_contratacao
Q48                  ==> transparencia_comunicacao
Q27                  ==> uso_limite_conta_corr_12_meses
Q29                  ==>

# Salvando

In [153]:
df_transf.to_excel('../data/Pesquisa_Credito_PF_Cols_Renomeada.xlsx')

# Transpondo colunas multivaloradas

In [154]:
def explodeColumns(df, affix=None, columnsSelector=None, columnRenamer=None, drop=True, var_name=None, value_name='valor', addMultValor=False):
    if columnsSelector:
        selectedCols = columnsSelector(df.columns);
    elif affix:
        selectedCols = [c for c in df.columns if affix in c]
    else:
        return df

    df = df[['id']+selectedCols].copy()

    if columnRenamer:
        df = (df.rename(columns={cc:columnRenamer(cc) for cc in selectedCols}))

    df =  df.melt(id_vars='id', var_name=var_name, value_name=value_name)

    if addMultValor:
        df = df.assign(categoria=affix)

    return df, selectedCols

In [155]:
def processaNPS(df_transf2):
    selectedCols_nps = []
    perguntas = [
        {'affix':'cartao_credito','replacer':'cartao_credito__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'endividamento','replacer':'endividamento__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'personalizacao_cobranca','replacer':'personalizacao_cobranca__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'atendimento_robo','replacer':'atendimento_robo__', 'var_name':'pergunta', 'value_name':'nps'},
        
        {'affix':'planejamento_credito','replacer':'planejamento_credito__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'cheque_especial','replacer':'cheque_especial__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'jornada_contratacao_credito_principal','replacer':'jornada_contratacao_credito_principal__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'credito_negado','replacer':'credito_negado__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'capacidade_financeira','replacer':'capacidade_financeira__', 'var_name':'pergunta', 'value_name':'nps'},
        
        {'affix':'situacao_credito','replacer':'situacao_credito__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'credito_parcelado','replacer':'credito_parcelado__', 'var_name':'pergunta', 'value_name':'nps'},
        {'affix':'crediario_digital','replacer':'crediario_digital__', 'var_name':'pergunta', 'value_name':'nps'},
    ]

    
    df_nps_list = [];
    
    for pergunta in perguntas:
        df_nps, selectedCols = explodeColumns(
            df_transf2,
            affix=pergunta['affix'],
            var_name=pergunta['var_name'],
            value_name=pergunta['value_name'],
            columnsSelector=lambda cols: [x for x in cols if x.startswith(pergunta['affix']+'__') and not re.match(r'.*\d+$',x)],
            columnRenamer=lambda x:x.replace(pergunta['replacer'],''),
            addMultValor=True
        )
        df_nps_list.append( df_nps.dropna())
        selectedCols_nps+=selectedCols
    
    return pd.concat(df_nps_list), selectedCols_nps

In [156]:
def processaRanks(df_transf2):
    selectedCols_geral = []
    perguntas = [
        {'affix':'prioridadePagamentoRank','replacer':'prioridadePagamentoRank__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'canal_contratacao_prioridade','replacer':'canal_contratacao_prioridade__', 'var_name':'rank', 'value_name':'prioridade'},
        
        
        {'affix':'fator_escolher_banco_emprestimo','replacer':'fator_escolher_banco_emprestimo__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'cartao_credito_novo__importancia','replacer':'cartao_credito_novo__importancia__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'credito_parcelado_novo__importancia','replacer':'credito_parcelado_novo__importancia__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'inadimplencia__melhor_forma_de_saber','replacer':'inadimplencia__melhor_forma_de_saber__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'melhorias_processo_cobranca','replacer':'melhorias_processo_cobranca__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'tipo_ofertas_renegociacao','replacer':'tipo_ofertas_renegociacao__', 'var_name':'rank', 'value_name':'prioridade'},
        {'affix':'meio_pagamento_preferencial','replacer':'meio_pagamento_preferencial__', 'var_name':'rank', 'value_name':'prioridade'},
        
    ]

    
    df_list = [];
    
    for pergunta in perguntas:
        df_ranks, selectedCols = explodeColumns(
            df_transf2,
            affix=pergunta['affix'],
            var_name=pergunta['var_name'],
            value_name=pergunta['value_name'],
            columnsSelector=lambda cols: [x for x in cols if x.startswith(pergunta['affix']+'__')],
            columnRenamer=lambda x:x.replace(pergunta['replacer'],''),
            addMultValor=True
        )
        df_list.append( df_ranks.dropna())
        selectedCols_geral+=selectedCols
    
    return pd.concat(df_list), selectedCols_geral

In [157]:
def processaBoolean(df_transf2):
    selectedCols_geral = []
    perguntas = [
        {'affix':'origem_dinheiro_pagamento_contas','replacer':'origem_dinheiro_pagamento_contas__', 'var_name':'rank', 'value_name':'valor'},
        {'affix':'gerou_inadimplencia','replacer':'gerou_inadimplencia__', 'var_name':'rank', 'value_name':'valor'},
        {'affix':'impossibilitou_pagamento','replacer':'impossibilitou_pagamento__', 'var_name':'rank', 'value_name':'valor'},
        {'affix':'inadimplente_no_produto','replacer':'inadimplente_no_produto__', 'var_name':'rank', 'value_name':'valor'},
        {'affix':'fator_mudar_banco','replacer':'fator_mudar_banco__', 'var_name':'rank', 'value_name':'valor'},
        {'affix':'tendencias','replacer':'tendencias__', 'var_name':'rank', 'value_name':'valor'},
        
    ]

    
    df_list = [];
    
    for pergunta in perguntas:
        df_ranks, selectedCols = explodeColumns(
            df_transf2,
            affix=pergunta['affix'],
            var_name=pergunta['var_name'],
            value_name=pergunta['value_name'],
            columnsSelector=lambda cols: [x for x in cols if x.startswith(pergunta['affix']+'__')],
            columnRenamer=lambda x:x.replace(pergunta['replacer'],''),
            addMultValor=True
        )
        df_list.append( df_ranks.dropna())
        selectedCols_geral+=selectedCols
    
    return pd.concat(df_list), selectedCols_geral

In [158]:
df_transf2 = df_transf.copy()

columns2drop =[]

###############
df_depara = (
    pd.DataFrame(depara_colunas, columns=['Nome na pesquisa', 'Nome Renomeado'])
)

################ Conhece Banco
print('Conhece Banco')
df_conhece_banco, selectedCols = explodeColumns(
    df_transf2,
    affix='Conhece_Banco',
    var_name='banco',
    columnRenamer=lambda x:x.replace('Conhece_Banco__','')
)
columns2drop += selectedCols




################ Considera_Contracao
print('Considera_Contracao')
df_considera_contracao, selectedCols = explodeColumns(
    df_transf2,
    affix='Considera_Contracao',
    var_name='banco',
    columnRenamer=lambda x:x.replace('Considera_Contracao__','')
)
columns2drop += selectedCols

################ Possui conta
print('Possui conta')
df_PossuiConta, selectedCols = explodeColumns(
    df_transf2,
    affix='PossuiConta',
    var_name='banco',
    columnRenamer=lambda x:x.replace('PossuiConta__','')
)
columns2drop += selectedCols


################ produto banco
print('produto banco')
df_possui_produto_banco, selectedCols = explodeColumns(
    df_transf2,
    affix='ProdutoBanco',
    var_name='ProdutoBanco',
    columnRenamer=lambda x:x.replace('ProdutoBanco__','')
)
df_possui_produto_banco = (
    df_possui_produto_banco
    .query("valor == 'Sim'")
    .assign(ProdutoBanco=lambda x:x['ProdutoBanco'].str.split('__'))
    .assign(banco=lambda x:x['ProdutoBanco'].apply(lambda x:x[0]))
    .assign(produto=lambda x:x['ProdutoBanco'].apply(lambda x:x[1]))
    .drop(columns=['ProdutoBanco','valor'])
)
columns2drop += selectedCols


#### numero de cartoes por banco
print('numero de cartoes por banco')
df_n_cartoes_banco, selectedCols = explodeColumns(
    df_transf2,
    affix='n_cartoes_banco',
    var_name='ProdutoBanco',
    value_name='n_cartoes',
    columnRenamer=lambda x:x.replace('n_cartoes_banco__','')
)
df_n_cartoes_banco = df_n_cartoes_banco.dropna().query('n_cartoes >0')
columns2drop += selectedCols


#### Consolida perguntas do tipo NPS
print('columns NPS')
df_nps, selectedCols = processaNPS(df_transf2)
df_nps = df_nps.dropna()
columns2drop += selectedCols

#### Consolida perguntas do tipo NPS
print('Ranking Coluns')
df_ranks, selectedCols = processaRanks(df_transf2)
columns2drop += selectedCols

#### Consolida perguntas do tipo Booleans
# print('Ranking Booleans - Não serão deletadas')
# df_bools, selectedCols = processaBoolean(df_transf2)
# columns2drop += selectedCols

df_transf2.drop(columns=[c for c in columns2drop if c in df_transf2.columns], inplace=True)
print('Processo concluído')

Conhece Banco
Considera_Contracao
Possui conta
produto banco
numero de cartoes por banco
columns NPS
Ranking Coluns
Processo concluído


In [159]:
writer = pd.ExcelWriter("../data/Pesquisa Credito PF - compacta.xlsx", engine='xlsxwriter')
df_transf2.to_excel(writer,sheet_name='pesquisa')
df_depara.to_excel(writer,sheet_name='De-para colunas', index=False)
df_conhece_banco.to_excel(writer,sheet_name='conhece_banco', index=False)

# Salvando só quem tem conta
df_PossuiConta.query('valor == "Sim"').drop(columns='valor').to_excel(writer,sheet_name='possui_banco', index=False)
df_considera_contracao.dropna().to_excel(writer,  sheet_name='considera_contratar_no_banco', index=False)


# Salvando só quem tem produto no banco
df_possui_produto_banco.to_excel(writer,sheet_name='produto_por_banco', index=False)

####
df_n_cartoes_banco.to_excel(writer,sheet_name='cartoes_por_banco', index=False)

####
df_nps.to_excel(writer,sheet_name='questoes_nps', index=False)

####
df_ranks.to_excel(writer,sheet_name='questoes_ranking', index=False)

writer.close()
print('Processo concluído')

Processo concluído


In [160]:
#df_considera_contracao.dropna()

In [161]:
df_bools, selectedCols = processaBoolean(df_transf2)
df_bools.pivot_table(index=['categoria','rank'], columns=['valor'], values='id', aggfunc='nunique', dropna=True, margins=True)

valor                                                                                Aposentadoria  \
categoria                        rank                                                                
fator_mudar_banco                Credito_aprovado_com_maior_agilidade                          NaN   
                                 Maior_limite_de_credito_rotativo                              NaN   
                                 Maior_limite_para_financiamento                               NaN   
                                 Maior_prazo_ou_parcelas_mais_baratas_para_finan...            NaN   
                                 Nada                                                          NaN   
                                 Outros                                                        NaN   
                                 Taxas_menores_para_financiamento                              NaN   
gerou_inadimplencia              agua_luz                                                      NaN   
                                 aluguel_condominio                                            NaN   
                                 educacao                                                      NaN   
                                 material_construcao                                           NaN   
                                 moveis_e_eletro                                               NaN   
                                 saude                                                         NaN   
                                 supermercado_alimentacao                                      NaN   
                                 tv_internet_telefone                                          NaN   
                                 vestuario_calcados                                            NaN   
                                 viagens                                                       NaN   
impossibilitou_pagamento         autonomo_reducao_renda                                        NaN   
                                 desemprego_familia                                            NaN   
                                 despesa_aumentou                                              NaN   
                                 despesa_extraordinaria                                        NaN   
                                 despesa_saude                                                 NaN   
                                 emprestei_nome_terceiros                                      NaN   
                                 esquecimento                                                  NaN   
                                 salario_atrasado                                              NaN   
inadimplente_no_produto          carne_boleto                                                  NaN   
                                 cartao_credito                                                NaN   
                                 cartao_loja                                                   NaN   
                                 cheque_devolvido                                              NaN   
                                 cheque_especial                                               NaN   
                                 emprestimo_com_garantia                                       NaN   
                                 emprestimo_pessoal                                            NaN   
origem_dinheiro_pagamento_contas cobro_divida_de_terceiros                                     NaN   
                                 empresto_de_parentes                                          NaN   
                                 outros                                                      4.000   
                                 reduzo_custo                                                  NaN   
                                 renda_extra                                                   NaN   
                      

In [164]:
np.sort(df_pesquisa['QSPRINCIPAL'].unique())

array([201, 202, 203, 204, 205, 206, 208, 210, 211, 212, 216, 217, 218,
       219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 971, 972,
       973])

In [165]:
df_pesquisa['QSPRINCIPAL'].value_counts()

QSPRINCIPAL
222    301
217    192
206    176
203    173
211    169
226    141
971    114
216     78
973     41
219     39
224     36
223     30
210     25
972     21
228     11
221      9
230      9
220      7
205      7
218      7
227      5
201      3
204      2
212      1
202      1
208      1
225      1
Name: count, dtype: int64

In [163]:
df_transf2['tipo_emprego'].unique()

array(['Empregado tempo integral', 'Autonomo', 'Empregado tempo parcial',
       'Cuido da casa', 'Estudante', 'Aposentado', 'Desempregado',
       'Outros'], dtype=object)